In [70]:
import pandas as pd
import altair as alt
import datetime


In [2]:
df = pd.read_excel('2014-2020.xlsx')
df.head()

,Час/Дата,AES,TEC,VDE,TES,GES,GAES_GEN,CONSUMPTION,GAES_PUMP,UK_BLR_RUS,UK_EURO,UK_MLD,Unnamed: 12
0,24-31.12.2020,9235,2039,621,4942,385.0,0,16693,-405.0,-84.0,-11.0,-29.0,NaN
1,23-31.12.2020,9221,2159,707,5549,470.0,0,17805,0.0,-43.0,-212.0,-46.0,NaN
2,22-31.12.2020,9249,2377,709,5906,1000.0,0,18870,0.0,-13.0,-328.0,-30.0,NaN
3,21-31.12.2020,9256,2499,702,6329,909.0,322,19887,0.0,0.0,-64.0,-66.0,NaN
4,20-31.12.2020,9213,2521,702,6640,823.0,602,20387,0.0,-29.0,-48.0,-37.0,NaN


In [3]:
df.drop(['Unnamed: 12', 'UK_MLD',  'UK_EURO', 'UK_BLR_RUS', 'GAES_PUMP'], axis = 1, inplace = True)
df = df.rename(columns={'Час/Дата': 'Time/Date'})
df.head()


,Time/Date,AES,TEC,VDE,TES,GES,GAES_GEN,CONSUMPTION
0,24-31.12.2020,9235,2039,621,4942,385.0,0,16693
1,23-31.12.2020,9221,2159,707,5549,470.0,0,17805
2,22-31.12.2020,9249,2377,709,5906,1000.0,0,18870
3,21-31.12.2020,9256,2499,702,6329,909.0,322,19887
4,20-31.12.2020,9213,2521,702,6640,823.0,602,20387


In [4]:

df[['Hour','Date']] = df['Time/Date'].str.split('-',expand=True)
df[['Day','Month','Year']] = df['Date'].str.split('.',expand=True)

df.head()


,Time/Date,AES,TEC,VDE,TES,GES,GAES_GEN,CONSUMPTION,Hour,Date,Day,Month,Year
0,24-31.12.2020,9235,2039,621,4942,385.0,0,16693,24,31.12.2020,31,12,2020
1,23-31.12.2020,9221,2159,707,5549,470.0,0,17805,23,31.12.2020,31,12,2020
2,22-31.12.2020,9249,2377,709,5906,1000.0,0,18870,22,31.12.2020,31,12,2020
3,21-31.12.2020,9256,2499,702,6329,909.0,322,19887,21,31.12.2020,31,12,2020
4,20-31.12.2020,9213,2521,702,6640,823.0,602,20387,20,31.12.2020,31,12,2020


# 1 Як змінювалась структура генерації електроенергії за роками?
Так як у нас є не одна електростанція, сюди найкраще підходить stacked bar chart. Але не завжди доцільно використовувати саме цей спосіб (наприклад, якби у нас було 20 електростанцій).

In [5]:
df1 = df.copy(deep=False)
df1.drop(['CONSUMPTION'], axis = 1, inplace = True)
df1 = df1.groupby('Year').sum()
df1 = df1.reset_index()

df1.head()

,Year,AES,TEC,VDE,TES,GES,GAES_GEN
0,2014,88204418,14684731,1606386,68605877,7991074.0,839842
1,2015,87413763,12041074,1234936,50259819,4964478.0,1554857
2,2016,80762094,12841033,1093190,49879543,7113989.0,1621377
3,2017,85314258,12208062,1177032,44945303,8531952.0,1575099
4,2018,84351328,12261018,1863726,47747092,10072767.0,1564867


In [6]:
df1_m = df1.melt(id_vars=['Year'], var_name='Sources', value_name='Total')
df1_m.head()


,Year,Sources,Total
0,2014,AES,88204418.0
1,2015,AES,87413763.0
2,2016,AES,80762094.0
3,2017,AES,85314258.0
4,2018,AES,84351328.0


In [140]:

chart1 = alt.Chart(df1_m,title='Cтруктура генерації електроенергії за роками').mark_bar().encode(
    x = alt.X('Year:N'),
    y = alt.Y('Total:Q'),
    color = alt.Color('Sources:N'),
    tooltip = alt.Tooltip('Total')
)
chart1


alt.Chart(...)

# 2 Як залежить споживання електроенергії від дня року та години доби?

У цьому завданні я вирішила два підпункти зобразити на двох різних графіках - лінійному та точковому. Лінійний я вирішила використати, бо є плавна залежність, яку важче було б показати bar chart-ом, а точковий у другому підпункті - бо на графіку є багато точок, які "виділяються" і лінійний графік був би кострубатим.

In [8]:
df2 = df.copy(deep=False)
df2.drop(['Year', 'AES','TEC', 'VDE', 'TES', 'GES','GAES_GEN'], axis = 1, inplace = True)
df2.head()

,Time/Date,CONSUMPTION,Hour,Date,Day,Month
0,24-31.12.2020,16693,24,31.12.2020,31,12
1,23-31.12.2020,17805,23,31.12.2020,31,12
2,22-31.12.2020,18870,22,31.12.2020,31,12
3,21-31.12.2020,19887,21,31.12.2020,31,12
4,20-31.12.2020,20387,20,31.12.2020,31,12


In [9]:
df2.drop(['Time/Date'], axis = 1, inplace = True)
df2.head()

,CONSUMPTION,Hour,Date,Day,Month
0,16693,24,31.12.2020,31,12
1,17805,23,31.12.2020,31,12
2,18870,22,31.12.2020,31,12
3,19887,21,31.12.2020,31,12
4,20387,20,31.12.2020,31,12


In [11]:
df21 = df2.copy(deep=False)
df21.drop(['Day'], axis = 1, inplace = True)

df21 = df21.groupby("Hour").mean().round().reset_index()
df21.head()


,Hour,CONSUMPTION
0,1,15193.0
1,10,18075.0
2,11,18206.0
3,12,18124.0
4,13,18084.0


In [18]:
df21['Hour'] = df21['Hour'].astype('int')

df2.dtypes

CONSUMPTION     int64
Hour           object
Date           object
Day            object
Month          object
dtype: object

In [138]:
chart21 = alt.Chart(df21,title="Cпоживання електроенергії від години доби").mark_line().encode(
    x = alt.X('Hour:O'),
    y = alt.Y('CONSUMPTION:Q'))

chart21

alt.Chart(...)

In [116]:
df22 = df2.copy(deep=False)
df22["Day_Month"] = df22['Day'] + '.' + df22['Month']
df22.drop(['Hour'], axis = 1, inplace = True)

df22.head()

,CONSUMPTION,Date,Day,Month,Day_Month
0,16693,31.12.2020,31,12,31.12
1,17805,31.12.2020,31,12,31.12
2,18870,31.12.2020,31,12,31.12
3,19887,31.12.2020,31,12,31.12
4,20387,31.12.2020,31,12,31.12


In [117]:
df22 = df22.groupby("Day_Month").mean().round().reset_index()
df22['Day_Month'] = df22['Day_Month'].apply(lambda x: str(x.split(".", 1)[1])+"."+str(x.split(".", 1)[0]))
df22.head()

,Day_Month,CONSUMPTION
0,01.01,17750.0
1,01.02,20259.0
2,01.03,18930.0
3,01.04,17172.0
4,01.05,14260.0


In [139]:
chart22 = alt.Chart(df22, title="Cпоживання електроенергії від дня року").mark_point().encode(
    alt.X('Day_Month:Q',
        scale=alt.Scale(domain=(0, 13))
    ),
    y=alt.Y('CONSUMPTION:Q')
)
chart22

alt.Chart(...)

# 3 Як змінюється генерація електроенергії з різних джерел впродовж доби?

Як і у першому завданні я вирішила доцільним використати stacked bar chart, адже він найкраще передасть всю інформацію на одному зображенні.

In [136]:
df3 = df.copy(deep=False)
df3.drop(['Year', 'Time/Date','CONSUMPTION','GAES_GEN','Date','Day','Month'], axis = 1, inplace = True)
df3.head()

,AES,TEC,VDE,TES,GES,Hour
0,9235,2039,621,4942,385.0,24
1,9221,2159,707,5549,470.0,23
2,9249,2377,709,5906,1000.0,22
3,9256,2499,702,6329,909.0,21
4,9213,2521,702,6640,823.0,20


In [130]:
df3 = df3.groupby("Hour").mean().round().reset_index()
df3.head()

,Hour,AES,TEC,VDE,TES,GES
0,1,9534.0,1429.0,164.0,4883.0,414.0
1,10,9548.0,1501.0,530.0,5897.0,1088.0
2,11,9545.0,1499.0,623.0,5946.0,1072.0
3,12,9541.0,1496.0,675.0,5949.0,1003.0
4,13,9535.0,1493.0,676.0,5949.0,958.0


In [133]:
df3_m = df3.melt(id_vars=['Hour'], var_name='Sources', value_name='Total')
df3_m.head()

,Hour,Sources,Total
0,1,AES,9534.0
1,10,AES,9548.0
2,11,AES,9545.0
3,12,AES,9541.0
4,13,AES,9535.0


In [135]:
chart3 = alt.Chart(df3_m,title='Генерація електроенергії з різних джерел впродовж доби').mark_bar().encode(
    x = alt.X('Hour:N'),
    y = alt.Y('Total:Q'),
    color = alt.Color('Sources:N'),
    tooltip = alt.Tooltip('Total')
)
chart3

alt.Chart(...)

# 4. Як змінюється споживання електроенергії впродовж доби у розрізі місяців року та пір року?

# 5. Як змінюється споживання електроенергії впродовж тижня?
Так як у цьому завданні у нас є всього 7 значень по іксу, доречно використовувати звичайний bar chart.

In [81]:
df51 = pd.to_datetime(df['Date'])

In [82]:
df5 = df[['CONSUMPTION']]
df5['weekday'] = df51.dt.day_name()
df5['weekday_num'] = df51.dt.weekday

/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [83]:
df5 = df5.groupby('weekday').mean()
df5 = df5.reset_index()
df5

,weekday,CONSUMPTION,weekday_num
0,Friday,17306.823402,4.0
1,Monday,17168.106963,0.0
2,Saturday,16743.430137,5.0
3,Sunday,16192.543721,6.0
4,Thursday,17396.080487,3.0
5,Tuesday,17310.623288,1.0
6,Wednesday,17384.582992,2.0


In [84]:
chart5 = alt.Chart(df5,title="Зміна споживання електроенергії впродовж тижня").mark_bar().encode(
    alt.X('weekday:O', title='Day of week', sort = alt.Sort(field = 'weekday_num', order = 'ascending')),
    alt.Y('CONSUMPTION:Q', title='Consumption', 
          scale = alt.Scale(zero = False))
)

In [85]:
chart5

alt.Chart(...)